In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import os 

import requests
from io import StringIO
import yaml


In [ ]:
file_name = "may2024"

In [ ]:
# Function to load API keys from a YAML file
def load_api_keys(filepath):
    with open(filepath, 'r') as file:
        secrets = yaml.safe_load(file)
    return secrets

# Load the API keys
secrets_filepath = 'secrets.yaml'  # Path to your secrets.yaml file
secrets = load_api_keys(secrets_filepath)

# Access the API keys
api_url = secrets['redcap']['api_url']
redcap_PC_key = secrets['redcap']['redcap_PC_key']
redcap_SC_key = secrets['redcap']['redcap_SC_key']


In [ ]:
# Data payload for the request, specifying the report ID
data_sc = {
    'token': redcap_SC_key,
    'content': 'report',
    'format': 'csv',   
    'report_id': 119747, 
    'rawOrLabel': 'raw',  
    'rawOrLabelHeaders': 'raw',  
    'exportCheckboxLabel': 'false', 
    'returnFormat': 'json'
}

# Data payload for the request, specifying the report ID
data_pc = {
    'token': redcap_PC_key,
    'content': 'report',
    'format': 'csv',   
    'report_id': 119745,
    'rawOrLabel': 'raw',  
    'rawOrLabelHeaders': 'raw',  
    'exportCheckboxLabel': 'false',  
    'returnFormat': 'json'
}

# Make the POST request
response_sc = requests.post(api_url, data=data_sc)
response_pc = requests.post(api_url, data=data_pc)

# Check if the request was successful
if response_sc.status_code == 200 & response_pc.status_code == 200:
    df_sc = pd.read_csv(StringIO(response_sc.text))
    df_pc = pd.read_csv(StringIO(response_pc.text))
else:
    print(f"Failed to retrieve data: {response.status_code}")

In [ ]:
# PC
# #df = pd.read_csv('StagewiseImplementat-APIIMATAllItems_DATA_2024-02-19_1149.csv')

# #SC
# df = pd.read_csv('StagewiseImplementat-APIIMATAllItems_DATA_2024-03-05_0922.csv')

In [ ]:
c_type = 'SC'

In [ ]:
if c_type == 'SC':
    df = df_sc
else:
    df = df_pc

In [ ]:
df['program_id'].value_counts()

In [ ]:
# Specify the file path of the Excel file
# file_path = 'SITT-MAT Academy/SITT-MAT_IMAT-PC Profile by Sites_v.2.xlsx'
if c_type == 'PC':
    file_path = 'SITT-MAT_IMAT-PC Profile by Sites_No Graph_with LBC.xlsx'
else:
    file_path = 'SITT-MAT_IMAT-SC Profile by Sites_No Graph_with LBC.xlsx'

# Open the Excel file
workbook = openpyxl.load_workbook(filename=file_path)

# Get the sheet names within the workbook
sheet_names = workbook.sheetnames

# Print the sheet names
for sheet_name in sheet_names:
    print(sheet_name)

In [ ]:
df.head()

In [ ]:

# Load the Excel file
df_template = pd.read_excel('SITT-MAT_IMAT-PC Profile by Sites_No Graph_with LBC.xlsx')
df_template

In [ ]:
# keep only the columns we want
# Use list comprehension to preserve order and duplicates from list1
intersection = [item for item in df.columns if item in df_template.columns]

df = df[intersection]
# filter out nans
df = df[df['imat_d1_1'] == df['imat_d1_1']]

# only keep last survey of each id
df = df.groupby('program_id').last().reset_index()



In [ ]:
def clear_folder(folder_path):
    # Iterate over all the files and subdirectories in the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            # Delete the file
            os.remove(file_path)
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            # Delete the directory and its contents recursively
            os.rmdir(dir_path)


In [ ]:
folder_path = 'sheets_out/' + file_name
# Call the clear_folder function
clear_folder(folder_path)

In [ ]:

for index, row in df.iterrows():
    if row['imat_d1_1'] == row['imat_d1_1']:
        # Create a new workbook
        new_workbook = openpyxl.load_workbook(filename=file_path)

        # Get the sheet names within the workbook
        sheet_names = new_workbook.sheetnames
        # print(sheet_names)
        
        if c_type == 'PC':
            new_workbook.remove(new_workbook['Linked PC Summary Report Data'])
        else:
            new_workbook.remove(new_workbook['Linked SC Summary Report Data'])

        # new_workbook.remove(new_workbook['Sheet1'])
        
        if c_type == 'PC':
            # insert this new sheet and export 
            linked_sheet = new_workbook.create_sheet(title='Linked PC Summary Report Data')
        else:
            linked_sheet = new_workbook.create_sheet(title='Linked SC Summary Report Data')

        # Get the values from the DataFrame row
        row_values = row.values

        # Determine the row index to insert the values
        row_index = linked_sheet.max_row + 1
        
        # Write the row values to the sheet
        for col_num, value in enumerate(row_values, start=1):
            linked_sheet.cell(row=row_index, column=col_num).value = value

        # Move the new sheet to the first position
        new_workbook.move_sheet(linked_sheet, offset=-1)
        
        event_info = row['redcap_event_name'].split('_')
        # Save the workbook
        
        if c_type == 'PC':
            new_workbook.save(folder_path + '/SITT-MAT_IMAT-PC Profile - ' + row['program_id'] + '-' + event_info[0] + event_info[1] + '.xlsx')
        else:
            new_workbook.save(folder_path + '/SITT-MAT_IMAT-SC Profile - ' + row['program_id'] + '-' + event_info[0] + event_info[1] + '.xlsx')


In [ ]:
# !cp -r "sheets_out/" + file_name + "/SITT-MAT_IMAT-SC Profile - id45-nov2023.xlsx" "/Users/liachin-purcell/Library/CloudStorage/Box-Box/McGovern Lab/MAT for OUD/SITT-MAT/03. EMF/February 2024/IMAT Excels/SUD"